# Submitting tasks to different devices and monitoring tasks 

In this tutorial, we show how to submit circuits to different devices, and how to keep track of the tasks.

## Imports and setup

In [1]:
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

__NOTE__: Please enter your desired device and S3 location (bucket and key) below. If you are working with the local simulator ```LocalSimulator()``` you do not need to specify any S3 location. However, if you are using the managed cloud-based device or any QPU devices you need to specify the S3 location where your results will be stored. In this case, you need to replace the API call ```device.run(circuit, ...)``` below with ```device.run(circuit, s3_folder, ...)```. 

In [2]:
# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = f"amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

## Build a circuit

Let's build a simple circuit of a Bell state.

In [3]:
bell = Circuit().h(0).cnot(0, 1)
print(bell)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


## Specify the device and submit the task

Then we specify the device to run the circuit. For small or shallow circuits with up to 25 qubits, we recommend using the local simulator for rapid prototyping and testing. For large circuits with up to 34 qubits, you can use the fully-managed cloud-based simulator SV1. You can also run your circuit on one of the QPU devices. 

In [4]:
# set up the device to be the local simulator
# device = LocalSimulator()

# set up the device to be the managed simulator
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

# set up the device to be the Rigetti quantum computer
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")

# set up the device to be the IonQ quantum computer
# device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")

The code below submits the circuit to SV1 and obtains the result. With the ```run(...)``` method we need to provide the circuit, the S3 bucket and several parameters.

* ```shots``` refers to the number of measurement shots. Simulators support two simulation modes. For shots = 0, the simulator performs an exact simulation, returning the true values for all result types. For non-zero values of shots, the simulator samples from the output distribution to emulate the shot noise of real QPUs. QPU devices only allow shots > 0. 
* ```poll_timeout_seconds``` is the number of seconds you want to wait and poll the task before it times out; the default value is 5 days (that is $\sim 5*60*60*24$ seconds). 
* ```poll_interval_seconds``` is the frequency how often the task is polled, e.g., how often you call the Braket API to get the status; the default value is 1 second. 

In [5]:
# run the circuit (if you don't provide poll_timeout_seconds and poll_interval_seconds, the default values will apply)
task = device.run(bell, s3_folder, shots=100, poll_timeout_seconds=1000, poll_interval_seconds=1)

# or if you run the circuit on the local simulator, you only need to specify the circuit and shots
# task = device.run(bell, shots=100)

# get the result
result = task.result()
counts = result.measurement_counts
print(counts)

Counter({'11': 59, '00': 41})


The `task = device.run()` is an asynchronous operation. This means you can keep working while the system in the background polls for the results. When you call `task.result()`, this becomes a blocking call and the next cell will not run until the task is complete, additionally, you will get an error if the task is not complete within the timeout period.

Alternatively, you can use `async_result()` which is a non-blocking call. If you run ```async_result()```, the notebook immediately goes to the next cell without waiting for polling to complete. Calling `result()` on the `async_result` object before it has completed, an `asyncio.exceptions.InvalidStateError` will be raised. This is expected behavior. Later, you can call ```result()``` and get the actual result from the task. 

In [6]:
# example with async_result
async_result = device.run(bell, s3_folder, shots=100).async_result()

`async_result()` returns a Future object (Details on Future: https://docs.python.org/3.8/library/asyncio-future.html#asyncio.Future). You can define custom callbacks to be invoked when the Future is completed. 

In [7]:
# async_result returns back a Future. 
future = device.run(bell, s3_folder, shots=100).async_result()

# this is invoked when the Future is done. i.e. task is in a terminal state.
# This will print out to STDOUT when its done.
def call_back_function(future):
    print(f"Custom task Result: {future.result().measurement_probabilities}")

# attached the callback function to the future.
future.add_done_callback(call_back_function)

## Monitor a task

After a task is submitted, you can find the task id and check the status of the task at any time. You can also cancel a task using `cancel()`. Note that you cannot cancel a task once it starts to run.

In [8]:
task = device.run(bell, s3_folder, shots=100)

# get id and status of submitted task
task_id = task.id
status = task.state()
# print('ID of task:', task_id)
print('Status of task:', status)

# cancel task
task.cancel()
status = task.state()
print('Status of task:', status)

Status of task: CREATED
Status of task: CANCELLING


### Advanced logging

We can record the state of the task using advanced logging. You can transfer this code to a python script instead of a Jupyter notebook, and the script can run as a process in the background so that your laptop can go to sleep and the script will still run. These advanced logging techniques allow you to see the background polling and create a record for later debugging. 

In [9]:
import logging
from datetime import datetime

# set filename for logs
log_file = 'device_logs-'+datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')+'.txt'
print('Task info will be logged in:', log_file)

# create new logger object
logger = logging.getLogger("newLogger")

# configure to log to file device_logs.txt in the appending mode
logger.addHandler(logging.FileHandler(filename=log_file, mode='a'))

# add to file all log messages with level DEBUG or above
logger.setLevel(logging.DEBUG)

Task info will be logged in: device_logs-20200911113718.txt


In [10]:
# print logs
! cat {log_file}

Custom task Result: {'00': 0.56, '11': 0.44}
